In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 152
USE = 'clicks'

PART = 'A'

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
USR = np.load(f'test_user_{PART}.npy')
len( USR )

900625

In [4]:
LOAD = 709
OFFSET = ''

In [5]:
users = cudf.read_parquet(f'../../data/train_with_features/train_v{VER}_{PART}.pqt')
print( users.shape )
users.head()

(90062500, 5)


,user,item,pos,wgt,clicks
0,11098739,506879,4,10.495,0
1,11098739,259891,5,9.500,0
2,11098739,527968,6,9.460,0
3,11098739,1499305,7,7.990,0
4,11098739,253410,8,7.475,0


# Add Weights

In [6]:
for k in [1,3,4,5,6,7]:

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v564_v{k}.csv')
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v564_wgt_v{k}.csv')
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{k}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{k}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

1 (90062500, 6)
3 (90062500, 7)
4 (90062500, 8)
5 (90062500, 9)
6 (90062500, 10)
7 (90062500, 11)


In [7]:
#import gc
#users = users.to_pandas()
#gc.collect()

In [8]:
USE2 = 'orders'
OFFSET = 10
for j,k in enumerate(['../../data/candidate_scores/submission_v564_F',
                      '../../data/candidate_scores/submission_v612']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-6-22-otto/submission_v564_F (90062500, 12)
/raid/Kaggle/otto/data/submission_v612 (90062500, 13)


In [9]:
USE2 = 'clicks'
OFFSET = 12
for j,k in enumerate(['../../data/candidate_scores/submission_v564_F',
                      '../../data/candidate_scores/submission_v612']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-6-22-otto/submission_v564_F (90062500, 14)
/raid/Kaggle/otto/data/submission_v612 (90062500, 15)


In [10]:
#users = users.to_pandas()
#gc.collect()

In [11]:
if USE=='clicks': USE2 = 'orders'
else: USE2 = 'clicks'
    
OFFSET = 30
for j,k in enumerate(['../../data/candidate_scores/submission_v709']):

    dfA = pd.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.loc[dfA.user.isin(USR)].reset_index(drop=True)
    dfA = dfA.drop('session_type',axis=1)
    dfA = cudf.DataFrame(dfA)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(100),tmpA.shape[0]//100 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = pd.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.loc[dfB.user.isin(USR)].reset_index(drop=True)
    dfB = dfB.drop('session_type',axis=1)
    dfB = cudf.DataFrame(dfB)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(100),tmpB.shape[0]//100 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

/raid/Kaggle/otto/data0/submission_v709 (90062500, 16)


In [12]:
USE2 = 'orders'
OFFSET = 14

for j,k in enumerate( range(20,28) ):

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v564_v{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v564_wgt_v{k}.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

20 (90062500, 17)
21 (90062500, 18)
22 (90062500, 19)
23 (90062500, 20)
24 (90062500, 21)
25 (90062500, 22)
26 (90062500, 23)
27 (90062500, 24)


In [13]:
users = users.to_pandas()
gc.collect()

0

In [14]:
USE2 = 'orders'
OFFSET = 22
for j,k in enumerate(['../../data/candidate_scores/submission_v614']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    users = users.merge(dfA.to_pandas(),on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

/raid/Kaggle/otto/data/submission_v614 (90062500, 25)


In [15]:
USE2 = 'clicks'
OFFSET = 23
for j,k in enumerate(['../../data/candidate_scores/submission_v614']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    users = users.merge(dfA.to_pandas(),on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

/raid/Kaggle/otto/data/submission_v614 (90062500, 26)


In [16]:
#USE2 = 'orders'
OFFSET = 24

for j,(k,USE2) in enumerate( zip( range(31,34),['orders','orders','clicks'] ) ):

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v564_v{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v564_wgt_v{k}.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA.to_pandas(),on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

31 (90062500, 27)
32 (90062500, 28)
33 (90062500, 29)


In [17]:
#users = users.to_pandas()
#gc.collect()

In [18]:
users.head()

,user,item,pos,wgt,clicks,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26
0,11098557,1216721,14,8.713905,0,2.500000,0.928571,-3.0,-3.0,0.50,0.857143,0.10,4.785714,-3.0,3.900000,23.964285,-3.0,0.10,-3.0,-3.0,0.857143,0.928571,-3.0,-3.0,-3.0,-3.0,2.500000,0.50,2.200000
1,11098557,1812034,18,7.729045,0,2.464286,1.785714,-3.0,-3.0,0.45,0.571429,-3.00,5.271429,1.2,4.128572,17.821428,-3.0,-3.00,-3.0,-3.0,0.571429,1.785714,-3.0,-3.0,-3.0,-3.0,2.464286,0.45,1.228571
2,11098557,762181,20,7.502940,0,1.142857,0.678571,-3.0,-3.0,0.35,0.857143,0.10,3.028571,1.2,5.271429,15.721429,-3.0,0.10,-3.0,-3.0,0.857143,0.678571,-3.0,-3.0,-3.0,-3.0,1.142857,0.35,2.071429
3,11098557,1181243,22,7.465755,0,0.892857,1.678571,-3.0,-3.0,-3.00,0.857143,0.15,3.428571,1.2,3.771429,16.482143,-3.0,0.15,-3.0,-3.0,0.857143,1.678571,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,0.871429
4,11098557,1152897,19,7.676215,0,0.892857,0.892857,-3.0,-3.0,-3.00,0.571429,-3.00,2.357143,0.6,3.614286,10.753572,-3.0,-3.00,-3.0,-3.0,0.571429,0.892857,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,1.614286


# User and Item Features

In [19]:
item_features = pd.read_parquet('../../data/item_user_features/item12.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

(1825499, 7)


,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5
aid,,,,,,,
0,0.0,14,12,1.166667,0.0,1.660465,1.660465
1,0.052632,19,17,1.117647,0.229416,2.657424,2.657424
2,0.0,7,7,1.0,0.0,0.925820,0.925820
3,0.079235,732,392,1.867347,0.312539,82.133194,82.133194
4,0.0,45,37,1.216216,0.0,3.540245,3.540245


In [20]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')
del item_features
gc.collect()
users.head()

,user,item,pos,wgt,clicks,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5
0,11098557,1216721,14,8.713905,0,2.500000,0.928571,-3.0,-3.0,0.50,0.857143,0.10,4.785714,-3.0,3.900000,23.964285,-3.0,0.10,-3.0,-3.0,0.857143,0.928571,-3.0,-3.0,-3.0,-3.0,2.500000,0.50,2.200000,0.168246,422,263,1.604563,0.47014,12.990656,12.990656
1,11098557,1812034,18,7.729045,0,2.464286,1.785714,-3.0,-3.0,0.45,0.571429,-3.00,5.271429,1.2,4.128572,17.821428,-3.0,-3.00,-3.0,-3.0,0.571429,1.785714,-3.0,-3.0,-3.0,-3.0,2.464286,0.45,1.228571,0.241071,112,67,1.671642,0.541029,7.342635,7.342635
2,11098557,762181,20,7.502940,0,1.142857,0.678571,-3.0,-3.0,0.35,0.857143,0.10,3.028571,1.2,5.271429,15.721429,-3.0,0.10,-3.0,-3.0,0.857143,0.678571,-3.0,-3.0,-3.0,-3.0,1.142857,0.35,2.071429,0.05814,172,116,1.482759,0.280127,7.986894,7.986894
3,11098557,1181243,22,7.465755,0,0.892857,1.678571,-3.0,-3.0,-3.00,0.857143,0.15,3.428571,1.2,3.771429,16.482143,-3.0,0.15,-3.0,-3.0,0.857143,1.678571,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,0.871429,0.245968,248,128,1.9375,0.524678,40.373257,40.373257
4,11098557,1152897,19,7.676215,0,0.892857,0.892857,-3.0,-3.0,-3.00,0.571429,-3.00,2.357143,0.6,3.614286,10.753572,-3.0,-3.00,-3.0,-3.0,0.571429,0.892857,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,1.614286,0.104116,413,266,1.552632,0.336036,11.723805,11.723805


In [21]:
item_features = pd.read_parquet('../../data/item_user_features/item13.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

count_item_0_6 int32
count_item_1_6 int32
count_item_2_6 int32
count_item_3_6 int32
count_item_4_6 int32
count_item_5_6 int32
count_item_6_6 int32
count_user_0_6 int32
count_user_1_6 int32
count_user_2_6 int32
count_user_3_6 int32
count_user_4_6 int32
count_user_5_6 int32
count_user_6_6 int32
(874852, 23)


,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6
0,-1.000000,-1.0,0.000000,-1.000000,0.000000,0.000000,0.000000,-1.0,-1.0,2.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,0.755929,0.755929
2,0.000000,0.0,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,2.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,0.786796,0.786796
3,0.083333,0.0,-1.000000,0.236842,0.108696,0.047619,0.090909,60.0,18.0,-1.0,38.0,46.0,63.0,99.0,37.0,15.0,-1.0,16.0,25.0,39.0,54.0,32.294037,32.294037
4,-1.000000,-1.0,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.0,-1.0,1.0,2.0,-1.0,2.0,3.0,-1.0,-1.0,1.0,2.0,-1.0,2.0,1.0,1.214986,1.214986
11,-1.000000,-1.0,0.333333,-1.000000,-1.000000,-1.000000,0.000000,-1.0,-1.0,3.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.133893,1.133893


In [22]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')
del item_features
gc.collect()
users.head()

,user,item,pos,wgt,clicks,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6
0,11098557,1216721,14,8.713905,0,2.500000,0.928571,-3.0,-3.0,0.50,0.857143,0.10,4.785714,-3.0,3.900000,23.964285,-3.0,0.10,-3.0,-3.0,0.857143,0.928571,-3.0,-3.0,-3.0,-3.0,2.500000,0.50,2.200000,0.168246,422,263,1.604563,0.47014,12.990656,12.990656,0.200000,0.25,-1.000000,0.000000,0.285714,0.200000,0.333333,5.0,4.0,-1.0,9.0,7.0,5.0,6.0,4.0,3.0,-1.0,4.0,5.0,5.0,4.0,2.794553,2.794553
1,11098557,1812034,18,7.729045,0,2.464286,1.785714,-3.0,-3.0,0.45,0.571429,-3.00,5.271429,1.2,4.128572,17.821428,-3.0,-3.00,-3.0,-3.0,0.571429,1.785714,-3.0,-3.0,-3.0,-3.0,2.464286,0.45,1.228571,0.241071,112,67,1.671642,0.541029,7.342635,7.342635,1.000000,0.00,0.200000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,10.0,2.0,3.0,8.0,10.0,1.0,1.0,5.0,2.0,2.0,8.0,9.0,4.163332,4.163332
2,11098557,762181,20,7.502940,0,1.142857,0.678571,-3.0,-3.0,0.35,0.857143,0.10,3.028571,1.2,5.271429,15.721429,-3.0,0.10,-3.0,-3.0,0.857143,0.678571,-3.0,-3.0,-3.0,-3.0,1.142857,0.35,2.071429,0.05814,172,116,1.482759,0.280127,7.986894,7.986894,0.000000,-1.00,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,-1.0,2.0,1.0,2.0,3.0,1.0,1.0,-1.0,2.0,1.0,1.0,3.0,1.0,0.975900,0.975900
3,11098557,1181243,22,7.465755,0,0.892857,1.678571,-3.0,-3.0,-3.00,0.857143,0.15,3.428571,1.2,3.771429,16.482143,-3.0,0.15,-3.0,-3.0,0.857143,1.678571,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,0.871429,0.245968,248,128,1.9375,0.524678,40.373257,40.373257,0.222222,0.00,0.230769,0.285714,0.000000,0.307692,0.142857,9.0,2.0,13.0,7.0,3.0,13.0,7.0,6.0,2.0,8.0,6.0,3.0,8.0,6.0,4.347960,4.347960
4,11098557,1152897,19,7.676215,0,0.892857,0.892857,-3.0,-3.0,-3.00,0.571429,-3.00,2.357143,0.6,3.614286,10.753572,-3.0,-3.00,-3.0,-3.0,0.571429,0.892857,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,1.614286,0.104116,413,266,1.552632,0.336036,11.723805,11.723805,0.166667,0.20,0.000000,0.100000,0.000000,0.000000,0.250000,12.0,5.0,5.0,10.0,6.0,8.0,8.0,7.0,2.0,5.0,7.0,4.0,6.0,8.0,2.627691,2.627691


In [23]:
item_features = pd.read_parquet('../../data/item_user_features/item4.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

count_item int32
count_user int32
(1825499, 9)


,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
aid,,,,,,,,,
0,0.000000,33.0,26.0,54182.363281,425964.1875,1.269231,0.000000,18119.994141,142572.515625
1,0.031250,32.0,29.0,50521.250000,266521.2500,1.103448,0.176777,18789.619141,148777.078125
2,0.000000,12.0,12.0,42171.000000,286971.0000,1.000000,0.000000,11595.507812,129372.109375
3,0.079933,1201.0,668.0,50329.492188,321759.3125,1.797904,0.314015,16501.742188,167216.656250
4,0.035714,140.0,97.0,56197.734375,283306.3125,1.443299,0.186243,15681.410156,174638.421875


In [24]:
#users = users.to_pandas()

In [25]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')

In [26]:
users.head()

,user,item,pos,wgt,clicks,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
0,11098557,1216721,14,8.713905,0,2.500000,0.928571,-3.0,-3.0,0.50,0.857143,0.10,4.785714,-3.0,3.900000,23.964285,-3.0,0.10,-3.0,-3.0,0.857143,0.928571,-3.0,-3.0,-3.0,-3.0,2.500000,0.50,2.200000,0.168246,422,263,1.604563,0.47014,12.990656,12.990656,0.200000,0.25,-1.000000,0.000000,0.285714,0.200000,0.333333,5.0,4.0,-1.0,9.0,7.0,5.0,6.0,4.0,3.0,-1.0,4.0,5.0,5.0,4.0,2.794553,2.794553,0.174489,831.0,509.0,53424.875000,304827.03125,1.632613,0.473005,18047.892578,169584.234375
1,11098557,1812034,18,7.729045,0,2.464286,1.785714,-3.0,-3.0,0.45,0.571429,-3.00,5.271429,1.2,4.128572,17.821428,-3.0,-3.00,-3.0,-3.0,0.571429,1.785714,-3.0,-3.0,-3.0,-3.0,2.464286,0.45,1.228571,0.241071,112,67,1.671642,0.541029,7.342635,7.342635,1.000000,0.00,0.200000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,10.0,2.0,3.0,8.0,10.0,1.0,1.0,5.0,2.0,2.0,8.0,9.0,4.163332,4.163332,0.235955,267.0,154.0,54228.343750,366498.00000,1.733766,0.520753,17225.775391,176694.796875
2,11098557,762181,20,7.502940,0,1.142857,0.678571,-3.0,-3.0,0.35,0.857143,0.10,3.028571,1.2,5.271429,15.721429,-3.0,0.10,-3.0,-3.0,0.857143,0.678571,-3.0,-3.0,-3.0,-3.0,1.142857,0.35,2.071429,0.05814,172,116,1.482759,0.280127,7.986894,7.986894,0.000000,-1.00,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,-1.0,2.0,1.0,2.0,3.0,1.0,1.0,-1.0,2.0,1.0,1.0,3.0,1.0,0.975900,0.975900,0.086207,464.0,267.0,53218.281250,301432.06250,1.737828,0.323826,19134.363281,170059.937500
3,11098557,1181243,22,7.465755,0,0.892857,1.678571,-3.0,-3.0,-3.00,0.857143,0.15,3.428571,1.2,3.771429,16.482143,-3.0,0.15,-3.0,-3.0,0.857143,1.678571,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,0.871429,0.245968,248,128,1.9375,0.524678,40.373257,40.373257,0.222222,0.00,0.230769,0.285714,0.000000,0.307692,0.142857,9.0,2.0,13.0,7.0,3.0,13.0,7.0,6.0,2.0,8.0,6.0,3.0,8.0,6.0,4.347960,4.347960,0.154440,777.0,409.0,53311.042969,308285.56250,1.899755,0.423941,19849.113281,178740.312500
4,11098557,1152897,19,7.676215,0,0.892857,0.892857,-3.0,-3.0,-3.00,0.571429,-3.00,2.357143,0.6,3.614286,10.753572,-3.0,-3.00,-3.0,-3.0,0.571429,0.892857,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,1.614286,0.104116,413,266,1.552632,0.336036,11.723805,11.723805,0.166667,0.20,0.000000,0.100000,0.000000,0.000000,0.250000,12.0,5.0,5.0,10.0,6.0,8.0,8.0,7.0,2.0,5.0,7.0,4.0,6.0,8.0,2.627691,2.627691,0.096284,1184.0,692.0,53323.996094,316026.68750,1.710983,0.330250,20315.859375,169324.968750


In [27]:
del item_features
gc.collect()

0

In [28]:
item_features = pd.read_parquet('../../data/item_user_features/item10.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
users = users.merge(item_features, left_on='item', right_index=True, how='left')
users.head()

count_item3 int32
count_user3 int32
prev3 int32
next3 int32
orders3 int32
carts3 int32
(874852, 15)


,user,item,pos,wgt,clicks,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
0,11098557,1216721,14,8.713905,0,2.500000,0.928571,-3.0,-3.0,0.50,0.857143,0.10,4.785714,-3.0,3.900000,23.964285,-3.0,0.10,-3.0,-3.0,0.857143,0.928571,-3.0,-3.0,-3.0,-3.0,2.500000,0.50,2.200000,0.168246,422,263,1.604563,0.47014,12.990656,12.990656,0.200000,0.25,-1.000000,0.000000,0.285714,0.200000,0.333333,5.0,4.0,-1.0,9.0,7.0,5.0,6.0,4.0,3.0,-1.0,4.0,5.0,5.0,4.0,2.794553,2.794553,0.174489,831.0,509.0,53424.875000,304827.03125,1.632613,0.473005,18047.892578,169584.234375,0.194444,36.0,25.0,54540.140625,263340.12500,1.440000,0.401386,18838.333984,165664.187500,7.0,14.0,0.0,7.0,-1.0,1.166667
1,11098557,1812034,18,7.729045,0,2.464286,1.785714,-3.0,-3.0,0.45,0.571429,-3.00,5.271429,1.2,4.128572,17.821428,-3.0,-3.00,-3.0,-3.0,0.571429,1.785714,-3.0,-3.0,-3.0,-3.0,2.464286,0.45,1.228571,0.241071,112,67,1.671642,0.541029,7.342635,7.342635,1.000000,0.00,0.200000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,10.0,2.0,3.0,8.0,10.0,1.0,1.0,5.0,2.0,2.0,8.0,9.0,4.163332,4.163332,0.235955,267.0,154.0,54228.343750,366498.00000,1.733766,0.520753,17225.775391,176694.796875,0.085714,35.0,28.0,50234.457031,341525.87500,1.250000,0.284029,18094.484375,170153.781250,21.0,22.0,0.0,3.0,-1.0,1.500000
2,11098557,762181,20,7.502940,0,1.142857,0.678571,-3.0,-3.0,0.35,0.857143,0.10,3.028571,1.2,5.271429,15.721429,-3.0,0.10,-3.0,-3.0,0.857143,0.678571,-3.0,-3.0,-3.0,-3.0,1.142857,0.35,2.071429,0.05814,172,116,1.482759,0.280127,7.986894,7.986894,0.000000,-1.00,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,-1.0,2.0,1.0,2.0,3.0,1.0,1.0,-1.0,2.0,1.0,1.0,3.0,1.0,0.975900,0.975900,0.086207,464.0,267.0,53218.281250,301432.06250,1.737828,0.323826,19134.363281,170059.937500,0.000000,10.0,9.0,63179.398438,348299.40625,1.111111,0.000000,14261.355469,178256.718750,9.0,7.0,0.0,0.0,-1.0,-1.000000
3,11098557,1181243,22,7.465755,0,0.892857,1.678571,-3.0,-3.0,-3.00,0.857143,0.15,3.428571,1.2,3.771429,16.482143,-3.0,0.15,-3.0,-3.0,0.857143,1.678571,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,0.871429,0.245968,248,128,1.9375,0.524678,40.373257,40.373257,0.222222,0.00,0.230769,0.285714,0.000000,0.307692,0.142857,9.0,2.0,13.0,7.0,3.0,13.0,7.0,6.0,2.0,8.0,6.0,3.0,8.0,6.0,4.347960,4.347960,0.154440,777.0,409.0,53311.042969,308285.56250,1.899755,0.423941,19849.113281,178740.312500,0.222222,54.0,39.0,50023.183594,328423.18750,1.384615,0.419643,15162.596680,178501.953125,27.0,24.0,0.0,12.0,-1.0,1.000000
4,11098557,1152897,19,7.676215,0,0.892857,0.892857,-3.0,-3.0,-3.00,0.571429,-3.00,2.357143,0.6,3.614286,10.753572,-3.0,-3.00,-3.0,-3.0,0.571429,0.892857,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,1.614286,0.104116,413,266,1.552632,0.336036,11.723805,11.723805,0.166667,0.20,0.000000,0.100000,0.000000,0.000000,0.250000,12.0,5.0,5.0,10.0,6.0,8.0,8.0,7.0,2.0,5.0,7.0,4.0,6.0,8.0,2.627691,2.627691,0.096284,1184.0,692.0,53323.996094,316026.68750,1.710983,0.330250,20315.859375,169324.968750,0.111111,54.0,38.0,59729.351562,301329.34375,1.421053,0.371974,16590.537109,165053.968750,35.0,33.0,1.0,4.0,1.0,1.000000


In [29]:
del item_features
gc.collect()

0

In [30]:
user_features = pd.read_parquet('../../data/item_user_features/user4.pqt')
for c in user_features.columns:
    if str(user_features[c].dtype)=='int32':
        print(c, user_features[c].dtype)
        user_features[c] = user_features[c].astype('float32')
print( user_features.shape )
user_features.head()

count_item2 int32
unique_item2 int32
(1801251, 9)


,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
session,,,,,,,,,
11098528,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000
11098529,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000
11098530,0.166667,6.0,2.0,79665.835938,338865.84375,3.000000,0.408248,547.326355,547.326355
11098531,0.333333,24.0,11.0,79429.914062,338629.90625,2.181818,0.761387,174.236954,174.236954
11098532,0.000000,2.0,2.0,79598.500000,338798.50000,1.000000,0.000000,562.149902,562.149902


In [31]:
users = users.merge(user_features, left_on='user', right_index=True, how='left')

In [32]:
del user_features
gc.collect()

0

In [33]:
users.head()

,user,item,pos,wgt,clicks,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
0,11098557,1216721,14,8.713905,0,2.500000,0.928571,-3.0,-3.0,0.50,0.857143,0.10,4.785714,-3.0,3.900000,23.964285,-3.0,0.10,-3.0,-3.0,0.857143,0.928571,-3.0,-3.0,-3.0,-3.0,2.500000,0.50,2.200000,0.168246,422,263,1.604563,0.47014,12.990656,12.990656,0.200000,0.25,-1.000000,0.000000,0.285714,0.200000,0.333333,5.0,4.0,-1.0,9.0,7.0,5.0,6.0,4.0,3.0,-1.0,4.0,5.0,5.0,4.0,2.794553,2.794553,0.174489,831.0,509.0,53424.875000,304827.03125,1.632613,0.473005,18047.892578,169584.234375,0.194444,36.0,25.0,54540.140625,263340.12500,1.440000,0.401386,18838.333984,165664.187500,7.0,14.0,0.0,7.0,-1.0,1.166667,0.0,12.0,7.0,79454.835938,338654.84375,1.714286,0.0,131.29068,131.29068
1,11098557,1812034,18,7.729045,0,2.464286,1.785714,-3.0,-3.0,0.45,0.571429,-3.00,5.271429,1.2,4.128572,17.821428,-3.0,-3.00,-3.0,-3.0,0.571429,1.785714,-3.0,-3.0,-3.0,-3.0,2.464286,0.45,1.228571,0.241071,112,67,1.671642,0.541029,7.342635,7.342635,1.000000,0.00,0.200000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,10.0,2.0,3.0,8.0,10.0,1.0,1.0,5.0,2.0,2.0,8.0,9.0,4.163332,4.163332,0.235955,267.0,154.0,54228.343750,366498.00000,1.733766,0.520753,17225.775391,176694.796875,0.085714,35.0,28.0,50234.457031,341525.87500,1.250000,0.284029,18094.484375,170153.781250,21.0,22.0,0.0,3.0,-1.0,1.500000,0.0,12.0,7.0,79454.835938,338654.84375,1.714286,0.0,131.29068,131.29068
2,11098557,762181,20,7.502940,0,1.142857,0.678571,-3.0,-3.0,0.35,0.857143,0.10,3.028571,1.2,5.271429,15.721429,-3.0,0.10,-3.0,-3.0,0.857143,0.678571,-3.0,-3.0,-3.0,-3.0,1.142857,0.35,2.071429,0.05814,172,116,1.482759,0.280127,7.986894,7.986894,0.000000,-1.00,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,-1.0,2.0,1.0,2.0,3.0,1.0,1.0,-1.0,2.0,1.0,1.0,3.0,1.0,0.975900,0.975900,0.086207,464.0,267.0,53218.281250,301432.06250,1.737828,0.323826,19134.363281,170059.937500,0.000000,10.0,9.0,63179.398438,348299.40625,1.111111,0.000000,14261.355469,178256.718750,9.0,7.0,0.0,0.0,-1.0,-1.000000,0.0,12.0,7.0,79454.835938,338654.84375,1.714286,0.0,131.29068,131.29068
3,11098557,1181243,22,7.465755,0,0.892857,1.678571,-3.0,-3.0,-3.00,0.857143,0.15,3.428571,1.2,3.771429,16.482143,-3.0,0.15,-3.0,-3.0,0.857143,1.678571,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,0.871429,0.245968,248,128,1.9375,0.524678,40.373257,40.373257,0.222222,0.00,0.230769,0.285714,0.000000,0.307692,0.142857,9.0,2.0,13.0,7.0,3.0,13.0,7.0,6.0,2.0,8.0,6.0,3.0,8.0,6.0,4.347960,4.347960,0.154440,777.0,409.0,53311.042969,308285.56250,1.899755,0.423941,19849.113281,178740.312500,0.222222,54.0,39.0,50023.183594,328423.18750,1.384615,0.419643,15162.596680,178501.953125,27.0,24.0,0.0,12.0,-1.0,1.000000,0.0,12.0,7.0,79454.835938,338654.84375,1.714286,0.0,131.29068,131.29068
4,11098557,1152897,19,7.676215,0,0.892857,0.892857,-3.0,-3.0,-3.00,0.571429,-3.00,2.357143,0.6,3.614286,10.753572,-3.0,-3.00,-3.0,-3.0,0.571429,0.892857,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,1.614286,0.104116,413,266,1.552632,0.336036,11.723805,11.723805,0.166667,0.20,0.000000,0.100000,0.000000,0.000000,0.250000,12.0,5.0,5.

In [34]:
for c in users.columns:
    print(c,users[c].dtype)

user int32
item int32
pos int32
wgt float32
clicks int8
score_1 float32
score_3 float32
score_4 float32
score_5 float32
score_6 float32
score_7 float32
score_10 float32
score_11 float32
score_12 float32
score_13 float32
score_30 float32
score_14 float32
score_15 float32
score_16 float32
score_17 float32
score_18 float32
score_19 float32
score_20 float32
score_21 float32
score_22 float32
score_23 float32
score_24 float32
score_25 float32
score_26 float32
buy_ratio_5 Float32
count_item_5 Int32
count_user_5 Int32
repeat_5 Float32
buy_ratio_std_5 Float32
item_day_std_5 float32
user_day_std_5 float32
buy_ratio_0_6 float32
buy_ratio_1_6 float32
buy_ratio_2_6 float32
buy_ratio_3_6 float32
buy_ratio_4_6 float32
buy_ratio_5_6 float32
buy_ratio_6_6 float32
count_item_0_6 float32
count_item_1_6 float32
count_item_2_6 float32
count_item_3_6 float32
count_item_4_6 float32
count_item_5_6 float32
count_item_6_6 float32
count_user_0_6 float32
count_user_1_6 float32
count_user_2_6 float32
count_user_3_

In [35]:
for c in users.columns:
    t = str(users[c].dtype)
    if (t=='int64'): 
        users[c] = users[c].astype('int32')
        print('convert',c,'to int32')
    if (t=='float64')|(t=='Int32')|(t=='Float32'):
        users[c] = users[c].astype('float32')
        print('convert',c,'to float32')

convert buy_ratio_5 to float32
convert count_item_5 to float32
convert count_user_5 to float32
convert repeat_5 to float32
convert buy_ratio_std_5 to float32


In [36]:
#%%time
#users = users.sort_values(['user','pos']).reset_index(drop=True)

In [37]:
#users = users.reset_index(drop=True)
#users = users.to_pandas()
#gc.collect()

In [38]:
users.head()

,user,item,pos,wgt,clicks,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
0,11098557,1216721,14,8.713905,0,2.500000,0.928571,-3.0,-3.0,0.50,0.857143,0.10,4.785714,-3.0,3.900000,23.964285,-3.0,0.10,-3.0,-3.0,0.857143,0.928571,-3.0,-3.0,-3.0,-3.0,2.500000,0.50,2.200000,0.168246,422.0,263.0,1.604563,0.470140,12.990656,12.990656,0.200000,0.25,-1.000000,0.000000,0.285714,0.200000,0.333333,5.0,4.0,-1.0,9.0,7.0,5.0,6.0,4.0,3.0,-1.0,4.0,5.0,5.0,4.0,2.794553,2.794553,0.174489,831.0,509.0,53424.875000,304827.03125,1.632613,0.473005,18047.892578,169584.234375,0.194444,36.0,25.0,54540.140625,263340.12500,1.440000,0.401386,18838.333984,165664.187500,7.0,14.0,0.0,7.0,-1.0,1.166667,0.0,12.0,7.0,79454.835938,338654.84375,1.714286,0.0,131.29068,131.29068
1,11098557,1812034,18,7.729045,0,2.464286,1.785714,-3.0,-3.0,0.45,0.571429,-3.00,5.271429,1.2,4.128572,17.821428,-3.0,-3.00,-3.0,-3.0,0.571429,1.785714,-3.0,-3.0,-3.0,-3.0,2.464286,0.45,1.228571,0.241071,112.0,67.0,1.671642,0.541029,7.342635,7.342635,1.000000,0.00,0.200000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,10.0,2.0,3.0,8.0,10.0,1.0,1.0,5.0,2.0,2.0,8.0,9.0,4.163332,4.163332,0.235955,267.0,154.0,54228.343750,366498.00000,1.733766,0.520753,17225.775391,176694.796875,0.085714,35.0,28.0,50234.457031,341525.87500,1.250000,0.284029,18094.484375,170153.781250,21.0,22.0,0.0,3.0,-1.0,1.500000,0.0,12.0,7.0,79454.835938,338654.84375,1.714286,0.0,131.29068,131.29068
2,11098557,762181,20,7.502940,0,1.142857,0.678571,-3.0,-3.0,0.35,0.857143,0.10,3.028571,1.2,5.271429,15.721429,-3.0,0.10,-3.0,-3.0,0.857143,0.678571,-3.0,-3.0,-3.0,-3.0,1.142857,0.35,2.071429,0.058140,172.0,116.0,1.482759,0.280127,7.986894,7.986894,0.000000,-1.00,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,-1.0,2.0,1.0,2.0,3.0,1.0,1.0,-1.0,2.0,1.0,1.0,3.0,1.0,0.975900,0.975900,0.086207,464.0,267.0,53218.281250,301432.06250,1.737828,0.323826,19134.363281,170059.937500,0.000000,10.0,9.0,63179.398438,348299.40625,1.111111,0.000000,14261.355469,178256.718750,9.0,7.0,0.0,0.0,-1.0,-1.000000,0.0,12.0,7.0,79454.835938,338654.84375,1.714286,0.0,131.29068,131.29068
3,11098557,1181243,22,7.465755,0,0.892857,1.678571,-3.0,-3.0,-3.00,0.857143,0.15,3.428571,1.2,3.771429,16.482143,-3.0,0.15,-3.0,-3.0,0.857143,1.678571,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,0.871429,0.245968,248.0,128.0,1.937500,0.524678,40.373257,40.373257,0.222222,0.00,0.230769,0.285714,0.000000,0.307692,0.142857,9.0,2.0,13.0,7.0,3.0,13.0,7.0,6.0,2.0,8.0,6.0,3.0,8.0,6.0,4.347960,4.347960,0.154440,777.0,409.0,53311.042969,308285.56250,1.899755,0.423941,19849.113281,178740.312500,0.222222,54.0,39.0,50023.183594,328423.18750,1.384615,0.419643,15162.596680,178501.953125,27.0,24.0,0.0,12.0,-1.0,1.000000,0.0,12.0,7.0,79454.835938,338654.84375,1.714286,0.0,131.29068,131.29068
4,11098557,1152897,19,7.676215,0,0.892857,0.892857,-3.0,-3.0,-3.00,0.571429,-3.00,2.357143,0.6,3.614286,10.753572,-3.0,-3.00,-3.0,-3.0,0.571429,0.892857,-3.0,-3.0,-3.0,-3.0,0.892857,-3.00,1.614286,0.104116,413.0,266.0,1.552632,0.336036,11.723805,11.723805,0.166667,0.20,0.000000,0.100000,0.000000,0.000

In [39]:
users.to_parquet(f'../../data/train_with_features/train_v{VER}_{PART}.pqt',index=False)